## Covid19 data aggregator

This notebook scrapes data from the best source of Covid19 data on the web and coverts it into a Pandas DataFrame

In [13]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import datetime

The best source of aggregate Covid-19 data i've found is [here](ncov2019.live/data); it is consolidated from various sources by [Avi Schiffmann](https://www.linkedin.com/in/avi-schiffmann/). 

Tremendous thanks to Avi for doing all of the scraping and consolidation code.

In [9]:
url = 'https://ncov2019.live/data'

We'll use a date stamp on the hour of query, I don't substanitive updates happening any more frequently than hourly.

In [10]:
now = datetime.datetime.now().strftime(("%Y-%m-%d:%H:00:00"))
now

'2020-03-12:08:00:00'

Data can be stored to disk and aggregated with other snapshots periodically to see trend data

In [11]:
directory = 'data'
storage_file = f'./{directory}/corona_{now}.pickle'

In [12]:
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')

tables = soup.find_all('tr')

covid_data_d = {}
for table in tables:
    table_bodies = table.find_all('td')
    region_data_l = []
    for table_body in table_bodies:
        region_data_l.append(table_body.get_text())

    if len(region_data_l) == 0:
        continue
    region_data_l = [i.lstrip().rstrip() for i in region_data_l]
    region_data_l = [i.replace(',','') for i in region_data_l]
    covid_data_d[region_data_l[0]] = region_data_l[1:]


raw_covid_df = pd.DataFrame(covid_data_d)
raw_covid_df

,Mainland China,Italy,Iran,South Korea,France,Spain,Germany,United States,Diamond Princess,Switzerland,...,Tasmania,External territories,Jervis Bay Territory,Canberra (ACT),Brasil,Panamá,México,Rep. Dominicana,Cuba,Bolivia
0,80793,12462,10075,7869,2281,2277,1966,1336,45,642,...,2,0,0,0,69,14,13,5,3,2
1,3169,827,429,66,48,55,3,36,0,4,...,0,0,0,0,0,1,0,0,0,0
2,62793,1045,2959,333,12,183,25,9,2,4,...,0,0,0,0,,,4,,,
3,4257,1028,,,105,126,,2,,,...,0,0,0,0,1,,1,,,


In [6]:
df = raw_covid_df.T.reset_index()
df['region'] = df['index']
del df['index']
df['Date'] = now
df = df.replace('',0)
df.columns = ['Confirmed', 'Deceased', 'Recovered', 'Serious', 'Region', 'Date']
df = df[['Date', 'Region', 'Confirmed', 'Deceased', 'Recovered', 'Serious']]         
df.set_index('Date', inplace = True)

In [7]:
df.to_pickle(storage_file)
df

,Region,Confirmed,Deceased,Recovered,Serious
Date,,,,,
2020-03-11:13:00:00,Mainland China,80778,3158,61475,4492
2020-03-11:13:00:00,Italy,10149,631,1004,877
2020-03-11:13:00:00,Iran,9000,354,2959,0
2020-03-11:13:00:00,South Korea,7755,60,288,0
2020-03-11:13:00:00,Spain,2124,49,136,126
...,...,...,...,...,...
2020-03-11:13:00:00,Newfoundland & Labrador,0,0,0,0
2020-03-11:13:00:00,Prince Edward Island,0,0,0,0
2020-03-11:13:00:00,Northwest Territories,0,0,0,0
